In [18]:
import pandas as pd
import numpy as np
import pymysql

import warnings

# Suprimir todos los warnings
warnings.filterwarnings("ignore")

# Suprimir los warnings de una categoría específica
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sqlalchemy import create_engine
from sqlalchemy import text

import getpass  # To get the password without showing the input
password = 'root'

In [19]:
bd = "inmigration_europe"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)

In [20]:
def asylum_applicants_unaccompanied(engine):
    with engine.connect() as connection:
        query = text(f'''SELECT * FROM asylum_applicants_unaccompanied''')
        result = connection.execute(query)
        # Fetch all rows from the result
        rows = result.fetchall()
        # Get column names from the result set
        columns = result.keys()
        # Create DataFrame with column names
        df_result = pd.DataFrame(rows, columns=columns)
    return df_result

In [21]:
df_result = asylum_applicants_unaccompanied(engine)
df_result

,citizen,sex,age,geo,TIME_PERIOD,OBS_VALUE
0,AF,F,Y16-17,NO,2008,5
1,AF,F,Y16-17,SE,2008,5
2,AF,F,Y16-17,UK,2008,5
3,AF,F,Y_LT14,AT,2008,5
4,AF,F,Y_LT14,EL,2008,5
...,...,...,...,...,...,...
14180,YE,M,Y16-17,DE,2023,25
14181,YE,M,Y16-17,EL,2023,10
14182,YE,M,Y16-17,FI,2023,5
14183,YE,M,Y16-17,NL,2023,135


In [22]:
df_result = df_result[df_result['citizen'] != 'EXT_EU27_2020 ']

In [23]:
df_result = df_result[df_result['citizen'] != 'EU27_2020']

In [24]:
df_result = df_result[df_result['citizen'] != 'TOTAL']

In [25]:
df_result['citizen'].value_counts()

citizen
AF    1111
SY     849
SO     835
IQ     584
ER     548
      ... 
KP       1
AR       1
BR       1
CL       1
DJ       1
Name: count, Length: 97, dtype: int64

In [26]:
# mapping = {
# 'Y_LT1' : '00_Y_LT1',
# 'Y1' : '01_Y1',
# 'Y2' : '02_Y2',
# 'Y3' : '03_Y3',
# 'Y4' : '04_Y4',
# 'Y5' : '05_Y5',
# 'Y6' : '06_Y6',
# 'Y7' : '07_Y7',
# 'Y8' : '08_Y8',
# 'Y9' : '09_Y9',
# 'Y10' : '10_Y10',
# 'Y11' : '11_Y11',
# 'Y12' : '12_Y12',
# 'Y13' : '13_Y13',
# 'Y14' : '14_Y14',
# 'Y15' : '15_Y15',
# 'Y16' : '16_Y16',
# 'Y17' : '17_Y17',
# }

mapping = {
'Y_LT14' : '00_Y_LT14',
'Y14-15' : '01_Y14-15',
'Y16-17' : '02_Y16-17',
}

# Reemplazamos los datos
df_result["age"] = df_result["age"].replace(mapping)
df_result

,citizen,sex,age,geo,TIME_PERIOD,OBS_VALUE
0,AF,F,02_Y16-17,NO,2008,5
1,AF,F,02_Y16-17,SE,2008,5
2,AF,F,02_Y16-17,UK,2008,5
3,AF,F,00_Y_LT14,AT,2008,5
4,AF,F,00_Y_LT14,EL,2008,5
...,...,...,...,...,...,...
14180,YE,M,02_Y16-17,DE,2023,25
14181,YE,M,02_Y16-17,EL,2023,10
14182,YE,M,02_Y16-17,FI,2023,5
14183,YE,M,02_Y16-17,NL,2023,135


In [27]:
df_result['age'].value_counts()

age
02_Y16-17    5987
01_Y14-15    2721
00_Y_LT14    1701
Name: count, dtype: int64

In [28]:
# Convertir la columna de años a datetime
df_result['TIME_PERIOD'] = pd.to_datetime(df_result['TIME_PERIOD'], format='%Y')

In [29]:
df_result = df_result[df_result['geo'] != 'UK']

In [30]:
geo_country_dict = {
    "AT": "Austria",
    "BE": "Belgium",
    "BG": "Bulgaria",
    "CY": "Cyprus",
    "CZ": "Czech Republic",
    "DE": "Germany",
    "DK": "Denmark",
    "EE": "Estonia",
    "EL": "Greece",
    "ES": "Spain",
    "FI": "Finland",
    "FR": "France",
    "HR": "Croatia",
    "HU": "Hungary",
    "IE": "Ireland",
    "IT": "Italy",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "LV": "Latvia",
    "MT": "Malta",
    "NL": "Netherlands",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SE": "Sweden",
    "SI": "Slovenia",
    "SK": "Slovakia",
    "CH": "Switzerland",
    "LI": "Liechtenstein",
    "NO": "Norway",
    "MK": "North Macedonia",
    "IS": "Iceland",
    "ME": "Montenegro"
}

df_result["geo"] = df_result["geo"].replace(geo_country_dict)

In [31]:
# Diccionario con los nombres de los países
country_dict = {
    'AD': 'Andorra', 'AE': 'United Arab Emirates', 'AF': 'Afghanistan', 'AG': 'Antigua and Barbuda', 'AL': 'Albania', 'AM': 'Armenia', 
    'AO': 'Angola', 'AR': 'Argentina', 'AT': 'Austria', 'AU': 'Australia', 'AZ': 'Azerbaijan', 'BA': 'Bosnia and Herzegovina', 
    'BB': 'Barbados', 'BD': 'Bangladesh', 'BE': 'Belgium', 'BF': 'Burkina Faso', 'BG': 'Bulgaria', 'BH': 'Bahrain', 'BI': 'Burundi', 
    'BJ': 'Benin', 'BL': 'Saint Barthelemy', 'BN': 'Brunei', 'BO': 'Bolivia', 'BR': 'Brazil', 'BS': 'Bahamas', 'BT': 'Bhutan', 
    'BW': 'Botswana', 'BY': 'Belarus', 'BZ': 'Belize', 'CA': 'Canada', 'CD': 'Congo (Kinshasa)', 'CF': 'Central African Republic', 
    'CG': 'Congo (Brazzaville)', 'CH': 'Switzerland', 'CI': 'Ivory Coast', 'CL': 'Chile', 'CM': 'Cameroon', 'CN': 'China', 'CO': 'Colombia', 
    'CR': 'Costa Rica', 'CU': 'Cuba', 'CV': 'Cape Verde', 'CY': 'Cyprus', 'CZ': 'Czech Republic', 'DE': 'Germany', 'DJ': 'Djibouti', 
    'DK': 'Denmark', 'DM': 'Dominica', 'DO': 'Dominican Republic', 'DZ': 'Algeria', 'EC': 'Ecuador', 'EE': 'Estonia', 'EG': 'Egypt', 
    'EH': 'Western Sahara', 'EL': 'Greece', 'ER': 'Eritrea', 'ES': 'Spain', 'ET': 'Ethiopia', 'FI': 'Finland', 'FJ': 'Fiji', 'FM': 'Micronesia', 
    'FR': 'France', 'GA': 'Gabon', 'GD': 'Grenada', 'GE': 'Georgia', 'GH': 'Ghana', 'GM': 'Gambia', 'GN': 'Guinea', 'GQ': 'Equatorial Guinea', 
    'GT': 'Guatemala', 'GW': 'Guinea-Bissau', 'GY': 'Guyana', 'HN': 'Honduras', 'HR': 'Croatia', 'HT': 'Haiti', 'HU': 'Hungary', 'ID': 'Indonesia', 
    'IE': 'Ireland', 'IL': 'Israel', 'IM': 'Isle of Man', 'IN': 'India', 'IQ': 'Iraq', 'IR': 'Iran', 'IS': 'Iceland', 'IT': 'Italy', 'JM': 'Jamaica', 
    'JO': 'Jordan', 'JP': 'Japan', 'KE': 'Kenya', 'KG': 'Kyrgyzstan', 'KH': 'Cambodia', 'KI': 'Kiribati', 'KM': 'Comoros', 'KN': 'Saint Kitts and Nevis', 
    'KP': 'North Korea', 'KR': 'South Korea', 'KW': 'Kuwait', 'KZ': 'Kazakhstan', 'LA': 'Laos', 'LB': 'Lebanon', 'LC': 'Saint Lucia', 'LI': 'Liechtenstein', 
    'LK': 'Sri Lanka', 'LR': 'Liberia', 'LS': 'Lesotho', 'LT': 'Lithuania', 'LU': 'Luxembourg', 'LV': 'Latvia', 'LY': 'Libya', 'MA': 'Morocco', 
    'MC': 'Monaco', 'MD': 'Moldova', 'ME': 'Montenegro', 'MG': 'Madagascar', 'MH': 'Marshall Islands', 'MK': 'North Macedonia', 'ML': 'Mali', 
    'MM': 'Myanmar', 'MN': 'Mongolia', 'MR': 'Mauritania', 'MT': 'Malta', 'MU': 'Mauritius', 'MV': 'Maldives', 'MW': 'Malawi', 'MX': 'Mexico', 
    'MY': 'Malaysia', 'MZ': 'Mozambique', 'NE': 'Niger', 'NG': 'Nigeria', 'NI': 'Nicaragua', 'NL': 'Netherlands', 'NO': 'Norway', 'NP': 'Nepal', 
    'NR': 'Nauru', 'NZ': 'New Zealand', 'OM': 'Oman', 'PA': 'Panama', 'PE': 'Peru', 'PG': 'Papua New Guinea', 'PH': 'Philippines', 'PK': 'Pakistan', 
    'PL': 'Poland', 'PS': 'Palestine', 'PT': 'Portugal', 'PW': 'Palau', 'PY': 'Paraguay', 'QA': 'Qatar', 'RO': 'Romania', 'RS': 'Serbia', 'RU': 'Russia', 
    'RW': 'Rwanda', 'SA': 'Saudi Arabia', 'SB': 'Solomon Islands', 'SC': 'Seychelles', 'SD': 'Sudan', 'SE': 'Sweden', 'SG': 'Singapore', 'SI': 'Slovenia', 
    'SK': 'Slovakia', 'SL': 'Sierra Leone', 'SM': 'San Marino', 'SN': 'Senegal', 'SO': 'Somalia', 'SR': 'Suriname', 'SS': 'South Sudan', 'ST': 'Sao Tome and Principe', 
    'SV': 'El Salvador', 'SY': 'Syria', 'SZ': 'Eswatini', 'TC': 'Turks and Caicos Islands', 'TD': 'Chad', 'TG': 'Togo', 'TH': 'Thailand', 'TJ': 'Tajikistan', 
    'TL': 'East Timor', 'TM': 'Turkmenistan', 'TN': 'Tunisia', 'TO': 'Tonga', 'TR': 'Turkey', 'TT': 'Trinidad and Tobago', 'TV': 'Tuvalu', 'TW': 'Taiwan', 
    'TZ': 'Tanzania', 'UA': 'Ukraine', 'UG': 'Uganda', 'UK': 'United Kingdom', 'US': 'United States', 'UY': 'Uruguay', 'UZ': 'Uzbekistan', 'VA': 'Vatican City', 
    'VC': 'Saint Vincent and the Grenadines', 'VE': 'Venezuela', 'VG': 'British Virgin Islands', 'VN': 'Vietnam', 'VU': 'Vanuatu', 'WF': 'Wallis and Futuna', 
    'WS': 'Samoa', 'XK': 'Kosovo', 'YE': 'Yemen', 'ZA': 'South Africa', 'ZM': 'Zambia', 'ZW': 'Zimbabwe', '' : 'STLS'
}

df_result["citizen"] = df_result["citizen"].replace(country_dict)

In [32]:
print(df_result['geo'].value_counts().to_string())

geo
Germany           1541
Sweden            1111
Belgium            758
Netherlands        728
Italy              713
Austria            676
Switzerland        581
France             578
Greece             502
Norway             409
Denmark            346
Finland            275
Bulgaria           203
Poland             176
Hungary            168
Slovenia           158
Cyprus             156
Romania            142
Luxembourg          76
Ireland             62
Croatia             60
Spain               47
Portugal            45
Malta               41
Slovakia            25
Lithuania           15
Latvia              10
Czech Republic       9
Iceland              6
Liechtenstein        3
Montenegro           1


In [33]:
print(df_result['citizen'].value_counts().to_string())

citizen
Afghanistan                 1069
Syria                        823
Somalia                      784
Iraq                         554
Eritrea                      488
Guinea                       357
Pakistan                     291
Morocco                      285
Iran                         245
Nigeria                      234
Algeria                      224
Congo (Kinshasa)             220
Ethiopia                     220
Russia                       216
STLS                         199
Albania                      181
Gambia                       178
UNK                          171
Turkey                       169
Ivory Coast                  154
Bangladesh                   132
Egypt                        130
Sierra Leone                 116
Tunisia                      114
Mali                         114
Libya                        104
Sudan                        101
Angola                        92
Cameroon                      89
Serbia                        88
Ko

In [34]:
df_result.to_csv('df_result_asylum_applicants_unaccompanied.csv',sep=';')